In [ ]:
import sys
import os

# Detect environment
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("Running locally or in Binder")

# Install UppASD if in Colab
if IN_COLAB:
    BRANCH_NAME = "python26"
    
    print(f"Installing UppASD from GitHub (branch: {BRANCH_NAME})...")
    print("This will take approximately 2-3 minutes...")
    
    # Clone repository
    !git clone --branch {BRANCH_NAME} https://github.com/UppASD/UppASD.git /content/UppASD
    
    # Build and install (prefer OpenBLAS on Colab)
    %cd /content/UppASD
    !mkdir -p build
    %cd build
    !BLA_VENDOR=OpenBLAS FC=gfortran cmake .. -DCMAKE_BUILD_TYPE=Release
    !make -j2
    %cd ..
    !BLA_VENDOR=OpenBLAS FC=gfortran pip install -e . --quiet
    
    # Return to content directory
    %cd /content
    
    print("✓ Installation complete!")
    print("✓ UppASD is ready to use")

## Cloud Environment Setup

This cell detects if you're running on Google Colab and installs UppASD if needed. It does nothing when running locally or on Binder (where UppASD is pre-installed).

# UppASD Interactive Simulation Notebook

This notebook provides a comprehensive environment for running **UppASD (Uppsala Atomic Spin Dynamics)** simulations and analyzing their results.

## Workflow
1. **Read Configuration**: Parse existing `inpsd.dat` input files
2. **Display System Info**: Show structural and simulation parameters
3. **Run Simulation**: Execute UppASD with the defined parameters
4. **Analyze Results**: Read and visualize magnetization and energy evolution
5. **Compare Results**: Overlay multiple simulations for comparative analysis

## Features
- Automated input file parsing
- Real-time simulation monitoring
- Publication-quality plots
- Multi-simulation comparison tools

## Section 1: Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import logging
from typing import Dict, List, Tuple, Optional

from uppasd.notebook import (
    read_inpsd,
    run_simulation_via_api,
    load_outputs,
    iteration_to_time,
    print_system_info,
    run_simulation_api,
    load_simulation_results,
)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Matplotlib settings
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## Section 2: Define File Reading Functions

## Section 3: Read and Parse inpsd.dat

In [ ]:
# Read the input file using uppasd.notebook helper
inpsd_path = 'inpsd.dat'  # Modify this path as needed
config = read_inpsd(inpsd_path)

logger.info(
    "Parsed inpsd.dat via uppasd.notebook: simid=%s, mode=%s, nstep=%s, T=%s",
    config.get('simid'), config.get('mode'), config.get('nstep'), config.get('temperature'),
)
logger.info("Input configuration loaded successfully")

## Section 4: Display System Information

In [ ]:
# Display configuration using packaged helper
print_system_info(config)

## Section 5: Run UppASD Simulation

In [ ]:
# Run UppASD via the packaged helper (uses the in-process API)
# sim_results = run_simulation_api()
# print(f"Energy: {sim_results['energy']}")

## Section 6: Read Simulation Output Files

In [ ]:
# Load results using packaged helper
results = load_simulation_results(config=config)
logger.info("Results loaded via uppasd.notebook")
print(f"\n✓ Results loaded successfully!")
if results['magnetization'] is not None:
    print(f"  Magnetization data: {results['magnetization'].shape[0]} points")
if results['energy'] is not None:
    print(f"  Energy data: {results['energy'].shape[0]} points")

## Section 7: Plot Magnetization vs Time

In [ ]:
def plot_magnetization(results: Dict, config: Dict = None, figsize: Tuple = (12, 6)) -> plt.Figure:
    """
    Plot magnetization evolution vs time/iteration.

    Parameters
    ----------
    results : dict
        Results dictionary from load_simulation_results
    config : dict, optional
        Configuration dictionary for additional info
    figsize : tuple
        Figure size

    Returns
    -------
    fig : matplotlib.figure.Figure
        The figure object
    """
    if results['magnetization'] is None or results['avg_time'] is None:
        logger.error("No magnetization data available")
        return None

    fig, ax = plt.subplots(figsize=figsize)

    time = results['avg_time']
    mag = results['magnetization']

    # Convert iteration to time if timestep is available
    if config and 'timestep' in config:
        time_physical = time * config['timestep']
        time_label = "Time (s)"
        ax_data = time_physical
    else:
        time_label = "Iteration"
        ax_data = time

    # Plot
    ax.plot(ax_data, mag, 'b-', linewidth=1.5, label='|<M>|')
    ax.fill_between(ax_data, mag, alpha=0.3)

    # Formatting
    ax.set_xlabel(time_label, fontsize=12, fontweight='bold')
    ax.set_ylabel('Average Magnetization (μ_B)', fontsize=12, fontweight='bold')
    ax.set_title(f"Magnetization Evolution - {config.get('simid', 'Simulation') if config else 'Simulation'}",
                 fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=11)

    # Add statistics
    m_final = mag[-1]
    m_initial = mag[0]
    m_mean = np.mean(mag)
    m_std = np.std(mag)

    stats_text = (f"Initial: {m_initial:.4f}\nFinal: {m_final:.4f}\n"
                  f"Mean: {m_mean:.4f} ± {m_std:.4f}")
    ax.text(0.98, 0.97, stats_text, transform=ax.transAxes,
            fontsize=10, verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.tight_layout()
    return fig


# Plot magnetization
if results['magnetization'] is not None:
    fig_mag = plot_magnetization(results, config)
    plt.show()
else:
    print("⚠️  No magnetization data available. Run simulation first or check output files.")

## Section 8: Plot Energy vs Time

In [ ]:
def plot_energy(results: Dict, config: Dict = None, figsize: Tuple = (12, 6)) -> plt.Figure:
    """
    Plot energy evolution vs time/iteration.

    Parameters
    ----------
    results : dict
        Results dictionary from load_simulation_results
    config : dict, optional
        Configuration dictionary for additional info
    figsize : tuple
        Figure size

    Returns
    -------
    fig : matplotlib.figure.Figure
        The figure object
    """
    if results['energy'] is None or results['ene_time'] is None:
        logger.error("No energy data available")
        return None

    fig, ax = plt.subplots(figsize=figsize)

    time = results['ene_time']
    energy = results['energy']

    # Convert iteration to time if timestep is available
    if config and 'timestep' in config:
        time_physical = time * config['timestep']
        time_label = "Time (s)"
        ax_data = time_physical
    else:
        time_label = "Iteration"
        ax_data = time

    # Plot total energy
    ax.plot(ax_data, energy, 'r-', linewidth=1.5, label='Total Energy', marker='o',
            markersize=3, markevery=max(1, len(energy)//50))

    # Formatting
    ax.set_xlabel(time_label, fontsize=12, fontweight='bold')
    ax.set_ylabel('Energy (Ry)', fontsize=12, fontweight='bold')
    ax.set_title(f"Energy Evolution - {config.get('simid', 'Simulation') if config else 'Simulation'}",
                 fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=11)

    # Add statistics
    e_initial = energy[0]
    e_final = energy[-1]
    e_mean = np.mean(energy)
    e_std = np.std(energy)
    e_drift = (e_final - e_initial) / abs(e_initial) * 100 if e_initial != 0 else 0

    stats_text = (f"Initial: {e_initial:.6f}\nFinal: {e_final:.6f}\n"
                  f"Mean: {e_mean:.6f} ± {e_std:.6f}\n"
                  f"Drift: {e_drift:.2f}%")
    ax.text(0.98, 0.97, stats_text, transform=ax.transAxes,
            fontsize=10, verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

    plt.tight_layout()
    return fig


def plot_energy_components(results: Dict, config: Dict = None, figsize: Tuple = (14, 8)) -> Optional[plt.Figure]:
    """
    Plot individual energy components vs time.

    Parameters
    ----------
    results : dict
        Results dictionary from load_simulation_results
    config : dict, optional
        Configuration dictionary for additional info
    figsize : tuple
        Figure size

    Returns
    -------
    fig : matplotlib.figure.Figure
        The figure object, or None if insufficient data
    """
    if results['totenergy_data'] is None or results['ene_time'] is None:
        logger.error("No energy component data available")
        return None

    data = results['totenergy_data']
    labels = results.get('totenergy_labels', [])

    if data.shape[1] < 3:
        logger.warning("Energy data has fewer than 3 columns (time + 2 components)")
        return None

    time = results['ene_time']

    # Convert iteration to time if timestep is available
    if config and 'timestep' in config:
        time_physical = time * config['timestep']
        time_label = "Time (s)"
    else:
        time_label = "Iteration"
        time_physical = time

    # Create subplots for energy components (typically: Tot, Exc, Ani, DM, PD, BiqDM, BQ, Dip, Zeeman, LSF, Chir, Ring)
    n_components = min(data.shape[1] - 1, 4)  # Plot up to 4 components
    fig, axes = plt.subplots(2, 2, figsize=figsize, sharex=True)
    axes = axes.flatten()

    component_names = ['Total', 'Exchange', 'Anisotropy', 'DM', 'PD', 'BiqDM', 'BQ', 'Dipolar', 'Zeeman', 'LSF', 'Chiral', 'Ring']

    for i in range(n_components):
        ax = axes[i]
        col = i + 1
        if col < data.shape[1]:
            comp_data = data[:, col]
            comp_name = component_names[i] if i < len(component_names) else f"Component {i}"

            ax.plot(time_physical, comp_data, 'o-', linewidth=2, markersize=3, markevery=max(1, len(comp_data)//30))
            ax.set_ylabel(f'{comp_name}\nEnergy (Ry)', fontsize=10, fontweight='bold')
            ax.grid(True, alpha=0.3)
            ax.set_title(f'{comp_name} Component', fontsize=11, fontweight='bold')

    axes[-1].set_xlabel(time_label, fontsize=12, fontweight='bold')
    axes[-2].set_xlabel(time_label, fontsize=12, fontweight='bold')

    fig.suptitle(f"Energy Components - {config.get('simid', 'Simulation') if config else 'Simulation'}",
                 fontsize=14, fontweight='bold', y=1.00)
    plt.tight_layout()
    return fig


# Plot energy
if results['energy'] is not None:
    fig_ene = plot_energy(results, config)
    plt.show()
else:
    print("⚠️  No energy data available. Check totenergy output file.")

# Plot energy components if available
if results['totenergy_data'] is not None:
    fig_ene_comp = plot_energy_components(results, config)
    if fig_ene_comp:
        plt.show()

## Section 9: Compare Multiple Simulation Results

In [ ]:
def compare_simulations(simids: List[str], figsize: Tuple = (14, 10)) -> None:
    """
    Compare magnetization and energy from multiple simulations.

    Parameters
    ----------
    simids : list
        List of simulation IDs to compare
    figsize : tuple
        Figure size
    """
    if not simids:
        logger.error("No simulation IDs provided")
        return

    # Load all results
    all_results = {}
    for simid in simids:
        all_results[simid] = load_simulation_results(simid)

    # Create comparison plots
    fig, axes = plt.subplots(2, 2, figsize=figsize)

    # Plot 1: Magnetization comparison
    ax = axes[0, 0]
    for simid, results in all_results.items():
        if results['magnetization'] is not None and results['avg_time'] is not None:
            ax.plot(results['avg_time'], results['magnetization'], 'o-', label=simid, markersize=3, markevery=max(1, len(results['avg_time'])//30))
    ax.set_xlabel('Iteration', fontsize=11, fontweight='bold')
    ax.set_ylabel('Magnetization |<M>| (μ_B)', fontsize=11, fontweight='bold')
    ax.set_title('Magnetization Comparison', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Plot 2: Energy comparison
    ax = axes[0, 1]
    for simid, results in all_results.items():
        if results['energy'] is not None and results['ene_time'] is not None:
            ax.plot(results['ene_time'], results['energy'], 's-', label=simid, markersize=3, markevery=max(1, len(results['ene_time'])//30))
    ax.set_xlabel('Iteration', fontsize=11, fontweight='bold')
    ax.set_ylabel('Total Energy (Ry)', fontsize=11, fontweight='bold')
    ax.set_title('Energy Comparison', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Plot 3: Final magnetization bar chart
    ax = axes[1, 0]
    final_mags = []
    sim_labels = []
    for simid, results in all_results.items():
        if results['magnetization'] is not None:
            final_mags.append(results['magnetization'][-1])
            sim_labels.append(simid)

    if final_mags:
        colors = plt.cm.viridis(np.linspace(0, 1, len(final_mags)))
        ax.bar(sim_labels, final_mags, color=colors, alpha=0.7, edgecolor='black')
        ax.set_ylabel('Final Magnetization (μ_B)', fontsize=11, fontweight='bold')
        ax.set_title('Final Magnetization Comparison', fontsize=12, fontweight='bold')
        ax.tick_params(axis='x', rotation=45)
        ax.grid(True, alpha=0.3, axis='y')

    # Plot 4: Final energy bar chart
    ax = axes[1, 1]
    final_enes = []
    sim_labels = []
    for simid, results in all_results.items():
        if results['energy'] is not None:
            final_enes.append(results['energy'][-1])
            sim_labels.append(simid)

    if final_enes:
        colors = plt.cm.plasma(np.linspace(0, 1, len(final_enes)))
        ax.bar(sim_labels, final_enes, color=colors, alpha=0.7, edgecolor='black')
        ax.set_ylabel('Final Energy (Ry)', fontsize=11, fontweight='bold')
        ax.set_title('Final Energy Comparison', fontsize=12, fontweight='bold')
        ax.tick_params(axis='x', rotation=45)
        ax.grid(True, alpha=0.3, axis='y')

    fig.suptitle('Multi-Simulation Comparison', fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

    # Print summary table
    print("\n" + "="*80)
    print(" SIMULATION COMPARISON SUMMARY ".center(80, "="))
    print("="*80)
    print(f"{'SimID':<20} {'Final |M|':<15} {'Final Energy':<15} {'ΔM':<15} {'ΔE':<15}")
    print("-"*80)

    for simid, results in all_results.items():
        m_final = results['magnetization'][-1] if results['magnetization'] is not None else np.nan
        m_init = results['magnetization'][0] if results['magnetization'] is not None else np.nan
        dm = m_final - m_init if not np.isnan(m_final) and not np.isnan(m_init) else np.nan

        e_final = results['energy'][-1] if results['energy'] is not None else np.nan
        e_init = results['energy'][0] if results['energy'] is not None else np.nan
        de = e_final - e_init if not np.isnan(e_final) and not np.isnan(e_init) else np.nan

        print(f"{simid:<20} {m_final:<15.6f} {e_final:<15.6f} {dm:<15.6f} {de:<15.6f}")

    print("="*80 + "\n")


# Example: Compare multiple simulations
# Uncomment and modify with your simulation IDs
# compare_simulations(['sim1', 'sim2', 'sim3'])